# Inteligencja Obliczeniowa w Analizie Danych Cyfrowych

##	Projekt I

### Autorzy
- Dominik Breksa
- Robert Barcik
- Konrad Bodzioch

Download all the necessary packages to run this `.ipynb` script.

Python version: `3.12.2`
Used packages:
- *numpy*, `1.26.4`
- *pandas*, `2.8.2`
- *matplotlib*, `3.8.3`
- *easyAI*, `2.0.12`

In [172]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install easyAI

Import all the necessary packages to run this `.ipynb` script.

In [173]:
import numpy as np
import pandas as pd

### Nimby

Nimby game created as described in the task. Probabilistic model added in make_move method.
Added typing to help with code completion and readability.

In [174]:
from easyAI.games import Nim
from typing import Optional, Final, ClassVar
from random import random

from easyAI import TwoPlayerGame, AI_Player


class Nimby(Nim):
    MUTATION_PROBABILITY: ClassVar[float] = 0.1
    
    def make_move(self, move: str) -> None:
        where, count = tuple(map(int, move.split(",")))
        #   Added randomness as described in the task
        if Nimby.MUTATION_PROBABILITY >= random():
            remove = count - 1
        else:
            remove = count

        self.piles[where - 1] -= remove

def ask_move_modified(self, game):
    # Make so that AI makes predictions as if the game was deterministic. This enables the AI to make mistakes based on randomness later, when game performs a move. Making sure that the AI doesn't predict wrong and build turn tree with wrong values.
    original_make_move = game.make_move
    game.make_move = Nim.make_move.__get__(game, Nim)
    move = self.AI_algo(game)
    game.make_move = original_make_move
    
    return move

# Dirty Hack, so that I don't have to create 3 classes (Nimby, Nim, TwoPlayerGame) once again to change one function.
AI_Player.ask_move = ask_move_modified

Make randomness seed deterministic.

In [175]:
from random import seed

seed(42)

Simple test of the Nimby game, to check if it works.

In [176]:
from easyAI import AI_Player, Negamax
from easyAI.AI import TranspositionTable

#   Test
ai1: Negamax = Negamax(8, tt=TranspositionTable())
ai2: Negamax = Negamax(4, tt=TranspositionTable())
nimby: Nimby = Nimby([AI_Player(ai1), AI_Player(ai2)])
nimby.play()

'player %d wins' % nimby.current_player

5 5 5 5

Move #1: player 1 plays 1,1 :
4 5 5 5

Move #2: player 2 plays 1,1 :
4 5 5 5

Move #3: player 1 plays 1,1 :
3 5 5 5

Move #4: player 2 plays 1,1 :
2 5 5 5

Move #5: player 1 plays 1,1 :
1 5 5 5

Move #6: player 2 plays 1,1 :
0 5 5 5

Move #7: player 1 plays 2,5 :
0 0 5 5

Move #8: player 2 plays 3,1 :
0 0 5 5

Move #9: player 1 plays 3,1 :
0 0 4 5

Move #10: player 2 plays 3,1 :
0 0 4 5

Move #11: player 1 plays 4,1 :
0 0 4 4

Move #12: player 2 plays 3,1 :
0 0 3 4

Move #13: player 1 plays 4,1 :
0 0 3 4

Move #14: player 2 plays 3,1 :
0 0 2 4

Move #15: player 1 plays 4,2 :
0 0 2 2

Move #16: player 2 plays 3,1 :
0 0 1 2

Move #17: player 1 plays 4,2 :
0 0 1 0

Move #18: player 2 plays 3,1 :
0 0 0 0


'player 1 wins'

### Negamax - Deterministic

1. Firstly we solve the Nim game using the `easyAI` library, in order to see max depth (`d == 14`) for perfect AI.
2. We can also acknowledge the fact that the first player will always lose (`w == -1`), and the perfect move is `1,1`

In [177]:
from easyAI import solve_with_iterative_deepening

from easyAI.games import Nim

perfect_table: TranspositionTable = TranspositionTable()
w, d, m = solve_with_iterative_deepening(Nim(), range(1, 25), win_score=80, tt=perfect_table)
w, d, m, len(perfect_table.d)

d:1, a:0, m:1,1
d:2, a:0, m:1,1
d:3, a:0, m:1,1
d:4, a:0, m:1,1
d:5, a:0, m:1,1
d:6, a:0, m:1,1
d:7, a:0, m:1,1
d:8, a:0, m:1,1
d:9, a:0, m:1,1
d:10, a:0, m:1,1
d:11, a:0, m:1,1
d:12, a:0, m:1,1
d:13, a:0, m:1,1
d:14, a:-100, m:1,1


(-1, 14, '1,1', 1100)

In [178]:
MAX_DEPTH: Final[int] = d

# Create a pandas dataframe to store the results
df: pd.DataFrame = pd.DataFrame(columns=['game_variant', 'algorithm', 'depths', 'starting_player', 'winner', 'time', 'rounds'])

# If you have the computing power you can test all the possible combinations of depths, since the game is deterministic, you will always get the same results.
# from itertools import product
# config: Final[list[tuple[int, int]]] = list(product(range(1, max_depth + 1), repeat=2))

# Tests as described in task.
CONFIG_NIM: Final[list[tuple[int, int]]] = [
    (MAX_DEPTH, MAX_DEPTH // 2),
    (MAX_DEPTH // 3, MAX_DEPTH // 4),
]

CONFIG_NIM

[(14, 7), (4, 3)]

In [179]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds


In [180]:
from typing import Callable, Any
from time import perf_counter

type DepthConfiguration = tuple[int, int]
type DataframeRecord = tuple[str, str, np.array, np.uint8, np.uint8, float, np.uint8]

def create_environment(*, game_type: type(TwoPlayerGame), solving_algorithm: Any, in_order: bool, **kwargs) -> Callable[[DepthConfiguration], DataframeRecord]:
    def play_game(depths: DepthConfiguration) -> DataframeRecord:
        player1_depth, player2_depth = depths
        ai_1: solving_algorithm = solving_algorithm(depth=player1_depth, tt=TranspositionTable())
        ai_2: solving_algorithm = solving_algorithm(depth=player2_depth, tt=TranspositionTable())
        game_config: dict[str, Any] = {
            'players': [
                AI_Player(ai_1),
                AI_Player(ai_2)
            ],
        } | kwargs
        
        environment: game_type = game_type(**game_config)
        
        if not in_order:
            environment.switch_player()
    
        starting_player: int = environment.current_player
    
        start: float = perf_counter()
        history: list = environment.play()
        end: float = perf_counter()

        output: DataframeRecord = str(game_type.__name__), str(solving_algorithm.__name__), np.asarray(depths), np.uint8(starting_player),  np.uint8(environment.current_player), end - start, np.uint8(len(history) - 1)

        print(
            '======== Finished(game_type=\'{}\', solving_algorithm=\'{}\')(depths={}, starting_player={}, winner={}, time={}s, rounds_number={}) ========'.format(
                *output
            )
        )
        return output
    
    return play_game

def add_to_dataframe(data: pd.DataFrame, records: list[DataframeRecord]) -> None:
    for record in records:
        data.loc[len(data)] = record

In [181]:
results: map = map(create_environment(game_type=Nim, solving_algorithm=Negamax, in_order=True), CONFIG_NIM)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 1 plays 1,2 :
3 5 5 5

Move #2: player 2 plays 1,1 :
2 5 5 5

Move #3: player 1 plays 2,3 :
2 2 5 5

Move #4: player 2 plays 1,1 :
1 2 5 5

Move #5: player 1 plays 2,1 :
1 1 5 5

Move #6: player 2 plays 1,1 :
0 1 5 5

Move #7: player 1 plays 2,1 :
0 0 5 5

Move #8: player 2 plays 3,1 :
0 0 4 5

Move #9: player 1 plays 4,1 :
0 0 4 4

Move #10: player 2 plays 3,2 :
0 0 2 4

Move #11: player 1 plays 4,2 :
0 0 2 2

Move #12: player 2 plays 4,2 :
0 0 2 0

Move #13: player 1 plays 3,1 :
0 0 1 0

Move #14: player 2 plays 3,1 :
0 0 0 0
======== Finished(game_type='Nim', solving_algorithm='Negamax')(depths=[14  7], starting_player=1, winner=1, time=0.8060025000013411s, rounds_number=14) ========
5 5 5 5

Move #1: player 1 plays 1,1 :
4 5 5 5

Move #2: player 2 plays 1,1 :
3 5 5 5

Move #3: player 1 plays 1,1 :
2 5 5 5

Move #4: player 2 plays 1,1 :
1 5 5 5

Move #5: player 1 plays 1,1 :
0 5 5 5

Move #6: player 2 plays 2,1 :
0 4 5 5

Move #7: player 1 plays 2,1 :
0 3 5 

In [182]:
results: map = map(create_environment(game_type=Nim, solving_algorithm=Negamax, in_order=False), CONFIG_NIM)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 2,1 :
4 4 5 5

Move #3: player 2 plays 1,1 :
3 4 5 5

Move #4: player 1 plays 3,3 :
3 4 2 5

Move #5: player 2 plays 1,1 :
2 4 2 5

Move #6: player 1 plays 4,1 :
2 4 2 4

Move #7: player 2 plays 1,1 :
1 4 2 4

Move #8: player 1 plays 3,1 :
1 4 1 4

Move #9: player 2 plays 2,1 :
1 3 1 4

Move #10: player 1 plays 4,1 :
1 3 1 3

Move #11: player 2 plays 1,1 :
0 3 1 3

Move #12: player 1 plays 3,1 :
0 3 0 3

Move #13: player 2 plays 2,1 :
0 2 0 3

Move #14: player 1 plays 4,1 :
0 2 0 2

Move #15: player 2 plays 4,1 :
0 2 0 1

Move #16: player 1 plays 2,2 :
0 0 0 1

Move #17: player 2 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nim', solving_algorithm='Negamax')(depths=[14  7], starting_player=2, winner=1, time=0.7052207999950042s, rounds_number=17) ========
5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 1,1 :
3 5 5 5

Move #3: player 2 plays 1,1 :
2 5 5 5

Move #4: player 1 plays 1,1 :
1 5

In [183]:
df.dtypes

game_variant        object
algorithm           object
depths              object
starting_player      uint8
winner               uint8
time               float64
rounds               uint8
dtype: object

In [184]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds
0,Nim,Negamax,"[14, 7]",1,1,0.806003,14
1,Nim,Negamax,"[4, 3]",1,2,0.139017,17
2,Nim,Negamax,"[14, 7]",2,1,0.705221,17
3,Nim,Negamax,"[4, 3]",2,1,0.098155,17


### Negamax - Non-Deterministic

In [185]:
from itertools import chain

REPEAT_COUNT: Final[int] = 10

CONFIG_NIMBY: Final[list[tuple[int, int]]] = list(chain.from_iterable([[element] * REPEAT_COUNT for element in CONFIG_NIM]))

CONFIG_NIMBY

[(14, 7),
 (14, 7),
 (14, 7),
 (14, 7),
 (14, 7),
 (14, 7),
 (14, 7),
 (14, 7),
 (14, 7),
 (14, 7),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3)]

In [186]:
results: map = map(create_environment(game_type=Nimby, solving_algorithm=Negamax, in_order=True), CONFIG_NIMBY)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 1 plays 1,2 :
3 5 5 5

Move #2: player 2 plays 1,1 :
3 5 5 5

Move #3: player 1 plays 3,2 :
3 5 3 5

Move #4: player 2 plays 1,1 :
2 5 3 5

Move #5: player 1 plays 2,1 :
2 4 3 5

Move #6: player 2 plays 1,1 :
1 4 3 5

Move #7: player 1 plays 3,3 :
1 4 0 5

Move #8: player 2 plays 1,1 :
0 4 0 5

Move #9: player 1 plays 4,1 :
0 4 0 5

Move #10: player 2 plays 4,1 :
0 4 0 5

Move #11: player 1 plays 4,1 :
0 4 0 4

Move #12: player 2 plays 2,1 :
0 3 0 4

Move #13: player 1 plays 4,1 :
0 3 0 3

Move #14: player 2 plays 4,1 :
0 3 0 2

Move #15: player 1 plays 2,1 :
0 2 0 2

Move #16: player 2 plays 2,2 :
0 0 0 2

Move #17: player 1 plays 4,1 :
0 0 0 1

Move #18: player 2 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nimby', solving_algorithm='Negamax')(depths=[14  7], starting_player=1, winner=1, time=0.8310328999941703s, rounds_number=18) ========
5 5 5 5

Move #1: player 1 plays 1,2 :
3 5 5 5

Move #2: player 2 plays 1,1 :
2 5 5 5

Move #3: player 1 plays 2,3 :


In [187]:
results: map = map(create_environment(game_type=Nimby, solving_algorithm=Negamax, in_order=False), CONFIG_NIMBY)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 2,1 :
4 4 5 5

Move #3: player 2 plays 1,1 :
3 4 5 5

Move #4: player 1 plays 3,3 :
3 4 2 5

Move #5: player 2 plays 1,1 :
3 4 2 5

Move #6: player 1 plays 1,3 :
0 4 2 5

Move #7: player 2 plays 3,1 :
0 4 1 5

Move #8: player 1 plays 3,1 :
0 4 0 5

Move #9: player 2 plays 4,1 :
0 4 0 4

Move #10: player 1 plays 2,1 :
0 3 0 4

Move #11: player 2 plays 4,1 :
0 3 0 3

Move #12: player 1 plays 4,1 :
0 3 0 3

Move #13: player 2 plays 2,1 :
0 2 0 3

Move #14: player 1 plays 4,1 :
0 2 0 2

Move #15: player 2 plays 4,2 :
0 2 0 0

Move #16: player 1 plays 2,1 :
0 1 0 0

Move #17: player 2 plays 2,1 :
0 0 0 0
======== Finished(game_type='Nimby', solving_algorithm='Negamax')(depths=[14  7], starting_player=2, winner=1, time=0.5453698000055738s, rounds_number=17) ========
5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 2,1 :
4 4 5 5

Move #3: player 2 plays 1,1 :
3 4 5 5

Move #4: player 1 plays 3,3 :
3

In [188]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds
0,Nim,Negamax,"[14, 7]",1,1,0.806003,14
1,Nim,Negamax,"[4, 3]",1,2,0.139017,17
2,Nim,Negamax,"[14, 7]",2,1,0.705221,17
3,Nim,Negamax,"[4, 3]",2,1,0.098155,17
4,Nimby,Negamax,"[14, 7]",1,1,0.831033,18
5,Nimby,Negamax,"[14, 7]",1,1,0.506849,16
6,Nimby,Negamax,"[14, 7]",1,1,0.492722,14
7,Nimby,Negamax,"[14, 7]",1,1,0.521217,16
8,Nimby,Negamax,"[14, 7]",1,2,0.499541,17
9,Nimby,Negamax,"[14, 7]",1,1,0.516394,16


### Negamax without alpha, beta pruning

In [189]:
from easyAI.AI.Negamax import inf

class NoPruningNegamax:

    def __init__(self, depth: int, scoring: Optional[Callable] = None, tt: Optional[TranspositionTable] = None) -> None:
        self.depth: int = depth
        self.scoring: Optional[Callable] = scoring
        self.tt: Optional[TranspositionTable] = tt

    def __call__(self, game: TwoPlayerGame) -> str:
        """
        Returns the AI's best move given the current state of the game.
        """

        scoring = (
            self.scoring if self.scoring else (lambda g: g.scoring())
        )  # horrible hack

        self.alpha = NoPruningNegamax.no_pruning_negamax(
            game,
            self.depth,
            self.depth,
            scoring,
            self.tt,
        )
        return game.ai_move # To jest dynamicznie dawana zmienna do obiektu klasy, bez konsultacji z faktyczną klasą gry. Rozwiązanie twórcy "easyAI" XD
    
    @staticmethod
    def no_pruning_negamax(game: TwoPlayerGame, depth: int, original_depth: int, scoring: Callable, tt: Optional[TranspositionTable] = None) -> float | int:
        """
		This implements Negamax with transposition tables.
		This method is not meant to be used directly. See ``easyAI.Negamax``
		for an example of practical use.
		This function is implemented (almost) according to
		http://en.wikipedia.org/wiki/Negamax
		"""
    
        # Is there a transposition table and is this game in it ?
        lookup = None if (tt is None) else tt.lookup(game)
    
        if lookup is not None:
            # The game has been visited in the past
    
            if lookup["depth"] >= depth:
                value = lookup["value"]
                if depth == original_depth:
                    game.ai_move = lookup["move"]
                return value
    
        if (depth == 0) or game.is_over():
            # NOTE: the "depth" variable represents the depth left to recurse into,
            # so the smaller it is, the deeper we are in the negamax recursion.
            # Here we add 0.001 as a bonus to signify that victories in less turns
            # have more value than victories in many turns (and conversely, defeats
            # after many turns are preferred over defeats in less turns)
            return scoring(game) * (1 + 0.001 * depth)
    
        if lookup is not None:
            # Put the supposedly best move first in the list
            possible_moves = game.possible_moves()
            possible_moves.remove(lookup["move"])
            possible_moves = [lookup["move"]] + possible_moves
    
        else:
            possible_moves = game.possible_moves()
    
        state = game
        best_move = possible_moves[0]
        if depth == original_depth:
            state.ai_move = possible_moves[0]
    
        best_value = -inf
        unmake_move = hasattr(state, "unmake_move")
    
        for move in possible_moves:
    
            if not unmake_move:
                game = state.copy()  # re-initialize move
    
            game.make_move(move)
            game.switch_player()
    
            move_score = -NoPruningNegamax.no_pruning_negamax(game, depth - 1, original_depth, scoring, tt)
    
            if unmake_move:
                game.switch_player()
                game.unmake_move(move)
    
            # bestValue = max( bestValue,  move_score )
            if best_value < move_score:
                best_value = move_score
                best_move = move
                if depth == original_depth:
                    state.ai_move = move
    
        if tt is not None:
    
            assert best_move in possible_moves
            tt.store(
                game=state,
                depth=depth,
                value=best_value,
                move=best_move,
            )
    
        return best_value


### No Pruning Negamax - Deterministic

In [190]:
results: map = map(create_environment(game_type=Nim, solving_algorithm=NoPruningNegamax, in_order=True), CONFIG_NIM)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 1 plays 1,1 :
4 5 5 5

Move #2: player 2 plays 1,1 :
3 5 5 5

Move #3: player 1 plays 3,2 :
3 5 3 5

Move #4: player 2 plays 1,1 :
2 5 3 5

Move #5: player 1 plays 3,1 :
2 5 2 5

Move #6: player 2 plays 1,1 :
1 5 2 5

Move #7: player 1 plays 3,1 :
1 5 1 5

Move #8: player 2 plays 1,1 :
0 5 1 5

Move #9: player 1 plays 3,1 :
0 5 0 5

Move #10: player 2 plays 2,1 :
0 4 0 5

Move #11: player 1 plays 4,1 :
0 4 0 4

Move #12: player 2 plays 2,1 :
0 3 0 4

Move #13: player 1 plays 4,1 :
0 3 0 3

Move #14: player 2 plays 4,1 :
0 3 0 2

Move #15: player 1 plays 2,1 :
0 2 0 2

Move #16: player 2 plays 4,1 :
0 2 0 1

Move #17: player 1 plays 2,2 :
0 0 0 1

Move #18: player 2 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nim', solving_algorithm='NoPruningNegamax')(depths=[14  7], starting_player=1, winner=1, time=1.0412022999953479s, rounds_number=18) ========
5 5 5 5

Move #1: player 1 plays 1,1 :
4 5 5 5

Move #2: player 2 plays 1,1 :
3 5 5 5

Move #3: player 1 plays

In [191]:
results: map = map(create_environment(game_type=Nim, solving_algorithm=NoPruningNegamax, in_order=False), CONFIG_NIM)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 3,1 :
4 5 4 5

Move #3: player 2 plays 1,1 :
3 5 4 5

Move #4: player 1 plays 3,1 :
3 5 3 5

Move #5: player 2 plays 1,1 :
2 5 3 5

Move #6: player 1 plays 3,1 :
2 5 2 5

Move #7: player 2 plays 1,1 :
1 5 2 5

Move #8: player 1 plays 3,1 :
1 5 1 5

Move #9: player 2 plays 1,1 :
0 5 1 5

Move #10: player 1 plays 3,1 :
0 5 0 5

Move #11: player 2 plays 2,1 :
0 4 0 5

Move #12: player 1 plays 4,1 :
0 4 0 4

Move #13: player 2 plays 2,1 :
0 3 0 4

Move #14: player 1 plays 4,1 :
0 3 0 3

Move #15: player 2 plays 4,1 :
0 3 0 2

Move #16: player 1 plays 2,1 :
0 2 0 2

Move #17: player 2 plays 4,1 :
0 2 0 1

Move #18: player 1 plays 2,2 :
0 0 0 1

Move #19: player 2 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nim', solving_algorithm='NoPruningNegamax')(depths=[14  7], starting_player=2, winner=1, time=0.8187510999996448s, rounds_number=19) ========
5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 play

In [192]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds
0,Nim,Negamax,"[14, 7]",1,1,0.806003,14
1,Nim,Negamax,"[4, 3]",1,2,0.139017,17
2,Nim,Negamax,"[14, 7]",2,1,0.705221,17
3,Nim,Negamax,"[4, 3]",2,1,0.098155,17
4,Nimby,Negamax,"[14, 7]",1,1,0.831033,18
5,Nimby,Negamax,"[14, 7]",1,1,0.506849,16
6,Nimby,Negamax,"[14, 7]",1,1,0.492722,14
7,Nimby,Negamax,"[14, 7]",1,1,0.521217,16
8,Nimby,Negamax,"[14, 7]",1,2,0.499541,17
9,Nimby,Negamax,"[14, 7]",1,1,0.516394,16


### No Pruning Negamax - Non-Deterministic

In [193]:
results: map = map(create_environment(game_type=Nimby, solving_algorithm=NoPruningNegamax, in_order=True), CONFIG_NIMBY)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 1 plays 1,1 :
4 5 5 5

Move #2: player 2 plays 1,1 :
4 5 5 5

Move #3: player 1 plays 3,1 :
4 5 4 5

Move #4: player 2 plays 1,1 :
3 5 4 5

Move #5: player 1 plays 3,1 :
3 5 3 5

Move #6: player 2 plays 1,1 :
2 5 3 5

Move #7: player 1 plays 3,1 :
2 5 2 5

Move #8: player 2 plays 1,1 :
1 5 2 5

Move #9: player 1 plays 3,1 :
1 5 1 5

Move #10: player 2 plays 1,1 :
0 5 1 5

Move #11: player 1 plays 3,1 :
0 5 1 5

Move #12: player 2 plays 2,1 :
0 4 1 5

Move #13: player 1 plays 3,1 :
0 4 0 5

Move #14: player 2 plays 4,1 :
0 4 0 4

Move #15: player 1 plays 4,1 :
0 4 0 3

Move #16: player 2 plays 2,1 :
0 3 0 3

Move #17: player 1 plays 4,1 :
0 3 0 2

Move #18: player 2 plays 2,1 :
0 2 0 2

Move #19: player 1 plays 4,1 :
0 2 0 1

Move #20: player 2 plays 2,2 :
0 0 0 1

Move #21: player 1 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nimby', solving_algorithm='NoPruningNegamax')(depths=[14  7], starting_player=1, winner=2, time=1.0925586000084877s, rounds_number=2

In [194]:
results: map = map(create_environment(game_type=Nimby, solving_algorithm=NoPruningNegamax, in_order=False), CONFIG_NIMBY)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 3,1 :
4 5 4 5

Move #3: player 2 plays 1,1 :
3 5 4 5

Move #4: player 1 plays 3,1 :
3 5 3 5

Move #5: player 2 plays 1,1 :
2 5 3 5

Move #6: player 1 plays 3,1 :
2 5 2 5

Move #7: player 2 plays 1,1 :
1 5 2 5

Move #8: player 1 plays 3,1 :
1 5 1 5

Move #9: player 2 plays 1,1 :
0 5 1 5

Move #10: player 1 plays 3,1 :
0 5 1 5

Move #11: player 2 plays 2,1 :
0 4 1 5

Move #12: player 1 plays 3,1 :
0 4 0 5

Move #13: player 2 plays 4,1 :
0 4 0 4

Move #14: player 1 plays 4,1 :
0 4 0 3

Move #15: player 2 plays 2,1 :
0 3 0 3

Move #16: player 1 plays 4,1 :
0 3 0 2

Move #17: player 2 plays 2,1 :
0 2 0 2

Move #18: player 1 plays 4,1 :
0 2 0 2

Move #19: player 2 plays 4,1 :
0 2 0 1

Move #20: player 1 plays 2,2 :
0 0 0 1

Move #21: player 2 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nimby', solving_algorithm='NoPruningNegamax')(depths=[14  7], starting_player=2, winner=1, time=0.8729795000108425s, rounds_number=2

In [195]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds
0,Nim,Negamax,"[14, 7]",1,1,0.806003,14
1,Nim,Negamax,"[4, 3]",1,2,0.139017,17
2,Nim,Negamax,"[14, 7]",2,1,0.705221,17
3,Nim,Negamax,"[4, 3]",2,1,0.098155,17
4,Nimby,Negamax,"[14, 7]",1,1,0.831033,18
...,...,...,...,...,...,...,...
83,Nimby,NoPruningNegamax,"[4, 3]",2,2,0.335432,20
84,Nimby,NoPruningNegamax,"[4, 3]",2,1,0.291607,21
85,Nimby,NoPruningNegamax,"[4, 3]",2,1,0.244109,17
86,Nimby,NoPruningNegamax,"[4, 3]",2,2,0.376971,22


## Expected-minimax

In [196]:
"""
The standard AI algorithm of easyAI is Negamax with alpha-beta pruning
and (optionally), transposition tables.
"""

LOWERBOUND, EXACT, UPPERBOUND = -1, 0, 1


def custom_negamax(game, depth, origDepth, scoring, alpha=+inf, beta=-inf, tt=None):
    """
    This implements Negamax with transposition tables.
    This method is not meant to be used directly. See ``easyAI.Negamax``
    for an example of practical use.
    This function is implemented (almost) according to
    http://en.wikipedia.org/wiki/Negamax
    """

    alphaOrig = alpha

    # Is there a transposition table and is this game in it ?
    lookup = None if (tt is None) else tt.lookup(game)

    if lookup is not None:
        # The game has been visited in the past

        if lookup["depth"] >= depth:
            flag, value = lookup["flag"], lookup["value"]
            if flag == EXACT:
                if depth == origDepth:
                    game.ai_move = lookup["move"]
                return value
            elif flag == LOWERBOUND:
                alpha = max(alpha, value)
            elif flag == UPPERBOUND:
                beta = min(beta, value)

            if alpha >= beta:
                if depth == origDepth:
                    game.ai_move = lookup["move"]
                return value

    if (depth == 0) or game.is_over():
        # NOTE: the "depth" variable represents the depth left to recurse into,
        # so the smaller it is, the deeper we are in the negamax recursion.
        # Here we add 0.001 as a bonus to signify that victories in less turns
        # have more value than victories in many turns (and conversely, defeats
        # after many turns are preferred over defeats in less turns)
        return scoring(game) * (1 + 0.001 * depth)

    if lookup is not None:
        # Put the supposedly best move first in the list
        possible_moves = game.possible_moves()
        possible_moves.remove(lookup["move"])
        possible_moves = [lookup["move"]] + possible_moves

    else:

        possible_moves = game.possible_moves()

    state = game
    best_move = possible_moves[0]
    if depth == origDepth:
        state.ai_move = possible_moves[0]

    bestValue = -inf
    unmake_move = hasattr(state, "unmake_move")

    for move in possible_moves:

        if not unmake_move:
            game = state.copy()  # re-initialize move

        game.make_move(move)
        game.switch_player()

        move_alpha = -custom_negamax(game, depth - 1, origDepth, scoring, -beta, -alpha, tt)

        if unmake_move:
            game.switch_player()
            game.unmake_move(move)

        # bestValue = max( bestValue,  move_alpha )
        if bestValue < move_alpha:
            bestValue = move_alpha
            best_move = move

        if alpha < move_alpha:
            alpha = move_alpha
            # best_move = move
            if depth == origDepth:
                state.ai_move = move
            if alpha >= beta:
                break

    if tt is not None:

        assert best_move in possible_moves
        tt.store(
            game=state,
            depth=depth,
            value=bestValue,
            move=best_move,
            flag=UPPERBOUND
            if (bestValue <= alphaOrig)
            else (LOWERBOUND if (bestValue >= beta) else EXACT),
        )

    return bestValue


class CustomNegamax:
    """
    This implements Negamax on steroids. The following example shows
    how to set up the AI and play a Connect Four game:

        >>> from easyAI.games import ConnectFour
        >>> from easyAI import Negamax, Human_Player, AI_Player
        >>> scoring = lambda game: -100 if game.lose() else 0
        >>> ai_algo = Negamax(8, scoring) # AI will think 8 turns in advance
        >>> game = ConnectFour([Human_Player(), AI_Player(ai_algo)])
        >>> game.play()

    Parameters
    -----------

    depth:
      How many moves in advance should the AI think ?
      (2 moves = 1 complete turn)

    scoring:
      A function f(game)-> score. If no scoring is provided
         and the game object has a ``scoring`` method it will be used.

    win_score:
      Score above which the score means a win. This will be
        used to speed up computations if provided, but the AI will not
        differentiate quick defeats from long-fought ones (see next
        section).

    tt:
      A transposition table (a table storing game states and moves)
      scoring: can be none if the game that the AI will be given has a
      ``scoring`` method.

    Notes
    -----

    The score of a given game is given by

    >>> scoring(current_game) - 0.01*sign*current_depth

    for instance if a loss is -100 points, then losing after 4 moves
    will score -99.96 points but losing after 8 moves will be -99.92
    points. Thus, the AI will choose the move that leads to defeat in
    8 turns, which makes it more difficult for the (human) opponent.
    This will not always work if a ``win_score`` argument is provided.

    """

    def __init__(self, depth: int, scoring=None, win_score=+inf, tt=None):
        self.scoring = scoring
        self.depth: int = depth
        self.tt = tt
        self.win_score = win_score

    def __call__(self, game):
        """
        Returns the AI's best move given the current state of the game.
        """

        scoring = (
            self.scoring if self.scoring else (lambda g: g.scoring())
        )  # horrible hack

        self.alpha = custom_negamax(
            game,
            self.depth,
            self.depth,
            scoring,
            -self.win_score,
            +self.win_score,
            self.tt,
        )
        return game.ai_move


### Expecti-minimax z odcięciem alfa-beta - Deterministic

In [197]:
results: map = map(create_environment(game_type=Nim, solving_algorithm=CustomNegamax, in_order=True), CONFIG_NIM)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 1 plays 1,2 :
3 5 5 5

Move #2: player 2 plays 1,1 :
2 5 5 5

Move #3: player 1 plays 2,3 :
2 2 5 5

Move #4: player 2 plays 1,1 :
1 2 5 5

Move #5: player 1 plays 2,1 :
1 1 5 5

Move #6: player 2 plays 1,1 :
0 1 5 5

Move #7: player 1 plays 2,1 :
0 0 5 5

Move #8: player 2 plays 3,1 :
0 0 4 5

Move #9: player 1 plays 4,1 :
0 0 4 4

Move #10: player 2 plays 3,2 :
0 0 2 4

Move #11: player 1 plays 4,2 :
0 0 2 2

Move #12: player 2 plays 4,2 :
0 0 2 0

Move #13: player 1 plays 3,1 :
0 0 1 0

Move #14: player 2 plays 3,1 :
0 0 0 0
======== Finished(game_type='Nim', solving_algorithm='CustomNegamax')(depths=[14  7], starting_player=1, winner=1, time=0.5505976999993436s, rounds_number=14) ========
5 5 5 5

Move #1: player 1 plays 1,1 :
4 5 5 5

Move #2: player 2 plays 1,1 :
3 5 5 5

Move #3: player 1 plays 1,1 :
2 5 5 5

Move #4: player 2 plays 1,1 :
1 5 5 5

Move #5: player 1 plays 1,1 :
0 5 5 5

Move #6: player 2 plays 2,1 :
0 4 5 5

Move #7: player 1 plays 2,1 :


In [198]:
results: map = map(create_environment(game_type=Nim, solving_algorithm=CustomNegamax, in_order=False), CONFIG_NIM)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 2,1 :
4 4 5 5

Move #3: player 2 plays 1,1 :
3 4 5 5

Move #4: player 1 plays 3,3 :
3 4 2 5

Move #5: player 2 plays 1,1 :
2 4 2 5

Move #6: player 1 plays 4,1 :
2 4 2 4

Move #7: player 2 plays 1,1 :
1 4 2 4

Move #8: player 1 plays 3,1 :
1 4 1 4

Move #9: player 2 plays 2,1 :
1 3 1 4

Move #10: player 1 plays 4,1 :
1 3 1 3

Move #11: player 2 plays 1,1 :
0 3 1 3

Move #12: player 1 plays 3,1 :
0 3 0 3

Move #13: player 2 plays 2,1 :
0 2 0 3

Move #14: player 1 plays 4,1 :
0 2 0 2

Move #15: player 2 plays 4,1 :
0 2 0 1

Move #16: player 1 plays 2,2 :
0 0 0 1

Move #17: player 2 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nim', solving_algorithm='CustomNegamax')(depths=[14  7], starting_player=2, winner=1, time=0.8313837999885436s, rounds_number=17) ========
5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 1,1 :
3 5 5 5

Move #3: player 2 plays 1,1 :
2 5 5 5

Move #4: player 1 plays 1,1

In [199]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds
0,Nim,Negamax,"[14, 7]",1,1,0.806003,14
1,Nim,Negamax,"[4, 3]",1,2,0.139017,17
2,Nim,Negamax,"[14, 7]",2,1,0.705221,17
3,Nim,Negamax,"[4, 3]",2,1,0.098155,17
4,Nimby,Negamax,"[14, 7]",1,1,0.831033,18
...,...,...,...,...,...,...,...
87,Nimby,NoPruningNegamax,"[4, 3]",2,1,0.282042,19
88,Nim,CustomNegamax,"[14, 7]",1,1,0.550598,14
89,Nim,CustomNegamax,"[4, 3]",1,2,0.139996,17
90,Nim,CustomNegamax,"[14, 7]",2,1,0.831384,17


### Expecti-minimax z odcięciem alfa-beta - Non-Deterministic

In [200]:
results: map = map(create_environment(game_type=Nimby, solving_algorithm=CustomNegamax, in_order=True), CONFIG_NIMBY)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 1 plays 1,2 :
3 5 5 5

Move #2: player 2 plays 1,1 :
2 5 5 5

Move #3: player 1 plays 2,3 :
2 2 5 5

Move #4: player 2 plays 1,1 :
1 2 5 5

Move #5: player 1 plays 2,1 :
1 1 5 5

Move #6: player 2 plays 1,1 :
0 1 5 5

Move #7: player 1 plays 2,1 :
0 0 5 5

Move #8: player 2 plays 3,1 :
0 0 4 5

Move #9: player 1 plays 4,1 :
0 0 4 4

Move #10: player 2 plays 3,2 :
0 0 2 4

Move #11: player 1 plays 4,2 :
0 0 2 2

Move #12: player 2 plays 4,2 :
0 0 2 0

Move #13: player 1 plays 3,1 :
0 0 1 0

Move #14: player 2 plays 3,1 :
0 0 0 0
======== Finished(game_type='Nimby', solving_algorithm='CustomNegamax')(depths=[14  7], starting_player=1, winner=1, time=0.580628900002921s, rounds_number=14) ========
5 5 5 5

Move #1: player 1 plays 1,2 :
3 5 5 5

Move #2: player 2 plays 1,1 :
2 5 5 5

Move #3: player 1 plays 2,3 :
2 2 5 5

Move #4: player 2 plays 1,1 :
1 2 5 5

Move #5: player 1 plays 2,1 :
1 1 5 5

Move #6: player 2 plays 1,1 :
0 1 5 5

Move #7: player 1 plays 2,1 :

In [201]:
results: map = map(create_environment(game_type=Nimby, solving_algorithm=CustomNegamax, in_order=False), CONFIG_NIMBY)
add_to_dataframe(df, list(results))

5 5 5 5

Move #1: player 2 plays 1,1 :
4 5 5 5

Move #2: player 1 plays 2,1 :
4 4 5 5

Move #3: player 2 plays 1,1 :
3 4 5 5

Move #4: player 1 plays 3,3 :
3 4 2 5

Move #5: player 2 plays 1,1 :
3 4 2 5

Move #6: player 1 plays 1,3 :
0 4 2 5

Move #7: player 2 plays 3,1 :
0 4 1 5

Move #8: player 1 plays 3,1 :
0 4 0 5

Move #9: player 2 plays 4,1 :
0 4 0 4

Move #10: player 1 plays 2,1 :
0 3 0 4

Move #11: player 2 plays 4,1 :
0 3 0 3

Move #12: player 1 plays 4,1 :
0 3 0 2

Move #13: player 2 plays 2,1 :
0 2 0 2

Move #14: player 1 plays 2,2 :
0 0 0 2

Move #15: player 2 plays 4,1 :
0 0 0 1

Move #16: player 1 plays 4,1 :
0 0 0 0
======== Finished(game_type='Nimby', solving_algorithm='CustomNegamax')(depths=[14  7], starting_player=2, winner=2, time=0.540254499996081s, rounds_number=16) ========
5 5 5 5

Move #1: player 2 plays 1,1 :
5 5 5 5

Move #2: player 1 plays 1,2 :
3 5 5 5

Move #3: player 2 plays 1,1 :
2 5 5 5

Move #4: player 1 plays 2,3 :
2 3 5 5

Move #5: player 2 plays 1,1

In [202]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds
0,Nim,Negamax,"[14, 7]",1,1,0.806003,14
1,Nim,Negamax,"[4, 3]",1,2,0.139017,17
2,Nim,Negamax,"[14, 7]",2,1,0.705221,17
3,Nim,Negamax,"[4, 3]",2,1,0.098155,17
4,Nimby,Negamax,"[14, 7]",1,1,0.831033,18
...,...,...,...,...,...,...,...
127,Nimby,CustomNegamax,"[4, 3]",2,2,0.106247,18
128,Nimby,CustomNegamax,"[4, 3]",2,2,0.153936,24
129,Nimby,CustomNegamax,"[4, 3]",2,1,0.112751,19
130,Nimby,CustomNegamax,"[4, 3]",2,1,0.115662,19


### Analysis

Added average time spend on computing the move, by all AI players.

In [203]:
df['avg_round_time'] = (df['time'] / df['rounds']).astype(np.float32)

In [204]:
print(f'Total computation time: {df["time"].sum()}s.')
print(f'Average game time: {df["time"].mean()}s.')
print(f'Average turn computation time: {df["avg_round_time"].mean()}s.')

Total computation time: 56.336277899870765s.
Average game time: 0.42678998408993s.
Average turn computation time: 0.02353229746222496s.


In [205]:
df

,game_variant,algorithm,depths,starting_player,winner,time,rounds,avg_round_time
0,Nim,Negamax,"[14, 7]",1,1,0.806003,14,0.057572
1,Nim,Negamax,"[4, 3]",1,2,0.139017,17,0.008177
2,Nim,Negamax,"[14, 7]",2,1,0.705221,17,0.041484
3,Nim,Negamax,"[4, 3]",2,1,0.098155,17,0.005774
4,Nimby,Negamax,"[14, 7]",1,1,0.831033,18,0.046168
...,...,...,...,...,...,...,...,...
127,Nimby,CustomNegamax,"[4, 3]",2,2,0.106247,18,0.005903
128,Nimby,CustomNegamax,"[4, 3]",2,2,0.153936,24,0.006414
129,Nimby,CustomNegamax,"[4, 3]",2,1,0.112751,19,0.005934
130,Nimby,CustomNegamax,"[4, 3]",2,1,0.115662,19,0.006087


In [206]:
PLAYER_1_NIM_WINS: Final[int] = len(df.where((df['winner'] == 1) & (df['game_variant'] == 'Nim')).dropna())
PLAYER_2_NIM_WINS: Final[int] = len(df.where((df['winner'] == 2) & (df['game_variant'] == 'Nim')).dropna())

In [207]:
print(f'Player 1 wins: {PLAYER_1_NIM_WINS} times in Nim games.')
print(f'Player 2 wins: {PLAYER_2_NIM_WINS} times in Nim games.')
print(f'Number of performed Nim games: {PLAYER_1_NIM_WINS + PLAYER_2_NIM_WINS}.')

Player 1 wins: 10 times in Nim games.
Player 2 wins: 2 times in Nim games.
Number of performed Nim games: 12.


In [208]:
PLAYER_1_NIMBY_WINS: Final[int] = len(df.where((df['winner'] == 1) & (df['game_variant'] == 'Nimby')).dropna())
PLAYER_2_NIMBY_WINS: Final[int] = len(df.where((df['winner'] == 2) & (df['game_variant'] == 'Nimby')).dropna())

In [209]:
print(f'Player 1 wins: {PLAYER_1_NIMBY_WINS} times in Nimby games.')
print(f'Player 2 wins: {PLAYER_2_NIMBY_WINS} times in Nimby games.')
print(f'Number of performed Nimby games: {PLAYER_1_NIMBY_WINS + PLAYER_2_NIMBY_WINS}.')

Player 1 wins: 68 times in Nimby games.
Player 2 wins: 52 times in Nimby games.
Number of performed Nimby games: 120.


In [210]:
print(f'Total number of games: {len(df)}.')

print(f'Player 1 wins: {PLAYER_1_NIM_WINS + PLAYER_1_NIMBY_WINS} times. Which is {((PLAYER_1_NIM_WINS + PLAYER_1_NIMBY_WINS) / len(df)) * 100:.2f}% of all games.')
print(f'Player 2 wins: {PLAYER_2_NIM_WINS + PLAYER_2_NIMBY_WINS} times. Which is {((PLAYER_2_NIM_WINS + PLAYER_2_NIMBY_WINS) / len(df)) * 100:.2f}% of all games.')

Total number of games: 132.
Player 1 wins: 78 times. Which is 59.09% of all games.
Player 2 wins: 54 times. Which is 40.91% of all games.


Save the results to a `.csv` file.

In [211]:
df.to_csv('results.csv', index=False)